<a href="https://colab.research.google.com/github/fathin123/BigData_Genap_2022/blob/main/Minggu12_BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# print working directory
!pwd

# List files and folders
!ls

# Check the open jdk version on colab
!ls /usr/lib/jvm/

/content
sample_data
default-java  java-1.11.0-openjdk-amd64  java-11-openjdk-amd64


In [4]:
# Download and install Java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,286 kB]
Hit:14 h

In [5]:
# Check if we have java 8 or not
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [6]:
# Download Apache Spark binary: This link can change based on the version. Update this link with the latest version before using
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

# Unzip file
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [8]:
# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 52.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f23044a81471291ae4a8a01435a9ce8b6cd23ec658ce3486570751febffb1cff
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [9]:
import findspark
findspark.init()

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
 .master("local")\
 .appName("Colab")\
 .config('spark.ui.port', '4050')\
 .getOrCreate()

In [11]:
!wget --continue https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json

--2022-06-06 07:30:34--  https://raw.githubusercontent.com/dhanifudin/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘sample_books.json’

sample_books.json   100%[===================>]   1.53K  --.-KB/s    in 0s      

2022-06-06 07:30:34 (23.2 MB/s) - ‘sample_books.json’ saved [1565/1565]



In [12]:
df = spark.read.json("/content/sample_books.json")

In [13]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [14]:
df.show(4,False)

+---------------+--------------+-----+----------------+------------+
|author         |edition       |price|title           |year_written|
+---------------+--------------+-----+----------------+------------+
|Austen, Jane   |Penguin       |18.2 |Northanger Abbey|1814        |
|Tolstoy, Leo   |Penguin       |12.7 |War and Peace   |1865        |
|Tolstoy, Leo   |Penguin       |13.5 |Anna Karenina   |1875        |
|Woolf, Virginia|Harcourt Brace|25.0 |Mrs. Dalloway   |1925        |
+---------------+--------------+-----+----------------+------------+
only showing top 4 rows



In [15]:
df.count()

13

In [16]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [17]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")

df_filtered.select("title", "price", "year_written").show(50, False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



# Tugas

1. Tampilkan data buku dengan harga paling murah!

In [18]:
df.show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Rowling, J.K.|   Har

In [19]:
df_filtered = df.filter("year_written IS NOT NULL AND price < 8 AND title IS NOT NULL")

df_filtered.select("title", "price", "year_written").show(50, False)

+-------------------------+-----+------------+
|title                    |price|year_written|
+-------------------------+-----+------------+
|Huckleberry Finn         |5.76 |1865        |
|Bleak House              |5.75 |1870        |
|Tom Sawyer               |7.75 |1862        |
|Hamlet, Prince of Denmark|7.95 |1603        |
+-------------------------+-----+------------+



2. Tampilkan jumlah terbit buku dikategorikan setiap tahun ditulis!

In [30]:
df_filtered = df.filter("year_written < 1900 AND price IS NOT NULL AND title IS NOT NULL")

df_filtered.select("title", "price", "year_written").count()


7

3. Tampilkan data buku termahal tiap tahun penulisannya!

In [31]:
df_filtered = df.filter("year_written IS NOT NULL AND price > 20 AND title IS NOT NULL")

df_filtered.select("title", "price", "year_written").show(50, False)

+-------------------+-----+------------+
|title              |price|year_written|
+-------------------+-----+------------+
|Mrs. Dalloway      |25.0 |1925        |
|A Room of One's Own|29.0 |1922        |
|Lord of the Rings  |27.45|1937        |
+-------------------+-----+------------+



4. Tampilkan data buku termurah tiap tahun penulisannya!

In [33]:
df_filtered = df.filter("year_written IS NOT NULL AND price < 7 AND title IS NOT NULL")

df_filtered.select("title", "price", "year_written").show(50, False)

+----------------+-----+------------+
|title           |price|year_written|
+----------------+-----+------------+
|Huckleberry Finn|5.76 |1865        |
|Bleak House     |5.75 |1870        |
+----------------+-----+------------+

